In [71]:
import pandas as pd
import os
from transformers import pipeline

In [ ]:
base_dir = os.getcwd()  # Get the script's directory
output_dir = os.path.join(base_dir, "..", "..", "output")

file_path1 = os.path.join(output_dir, "data_with_OCR_text.xlsx")
file_path2 = os.path.join(output_dir, "final_preprocessed_data.xlsx")
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

In [ ]:
df2 = df2.merge(df1[['Message_ID', 'Cleaned_Extracted_Text']], on='Message_ID', how='left')

In [ ]:
df2["Combined_text"] = df2["Message"].fillna('') + " " + df2["Cleaned_Extracted_Text"].fillna('') + " " + df2["Translated_Text"].fillna('')

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
labels = ["news", "non-news"]

In [ ]:
df2["Classification"] = df2["Combined_text"].apply(
    lambda x: classifier(x, labels)["labels"][0] if isinstance(x, str) and len(x.strip()) > 0 else "Unknown"
)

In [61]:
df2['Total_Comments'] = df2['Text_Comments'].fillna("") + df2['Message'].fillna("")

In [62]:
df2 = df2[df2['Classification'] != 'non-news']

In [63]:
df2.drop(columns=['Message', 'Text_Comments', 'Media File',
                  'Media_Comments', 'Translated_Text', 'Cleaned_Extracted_Text', 'Classification',
                  'Media_Positive_Reactions', 'Media_Negative_Reactions'], inplace=True)

In [65]:
cols = ['Message_ID', 'Date',  'Combined_text', 'Total_Comments', 'Text_Positive_Reactions', 'Text_Negative_Reactions']
df2 = df2[cols]

In [66]:
df2.head()

,Message_ID,Date,Combined_text,Total_Comments,Text_Positive_Reactions,Text_Negative_Reactions
0,13116,2025-01-02 23:28:31,"IDF: Over the past hour, terrorists hurled roc...","IDF: Over the past hour, terrorists hurled roc...",3,65
1,13115,2025-01-02 23:20:32,"🇺🇸🇮🇷⚡️- ""Biden discussed plans to strike Iran ...",['Biden/Obama are responsible for a lot of wha...,1,65
2,13114,2025-01-01 18:40:48,Former Israeli defence minister Yoav Gallant h...,['A Zionist Jew and an Anti Zionist Jew walk i...,80,0
3,13113,2025-01-01 18:40:38,"Gallant announced his retirement: ""There are m...","['Pls, do! U have obviously overstayed ur welc...",23,11
4,13112,2025-01-01 18:40:07,"🇵🇸🇮🇱⚡- ""If Hamas does not allow the release of...","['Welp... We know who you voted for!!', 'https...",0,27


In [70]:
file_path1 = os.path.join(output_dir, "auto_labeled_data.xlsx")
df2.to_excel(file_path1, index=False)